<a href="https://colab.research.google.com/github/cepdnaclk/e20-co542-Detect-AI-vs-Human-Generated-Images/blob/e20456/convnext_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
!pip install pandas

In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/competitions/detect-ai-vs-human-generated-images")
od.download("https://www.kaggle.com/datasets/alessandrasala79/ai-vs-human-generated-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: upsddulaj
Your Kaggle Key: ··········


100%|██████████| 1.89M/1.89M [00:00<00:00, 140MB/s]


Extracting archive ./detect-ai-vs-human-generated-images/detect-ai-vs-human-generated-images.zip to ./detect-ai-vs-human-generated-images
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

 upsddulaj
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/alessandrasala79/ai-vs-human-generated-dataset


100%|██████████| 9.76G/9.76G [01:15<00:00, 138MB/s]


In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
# Define paths to dataset files
path = '/content/ai-vs-human-generated-dataset'
train_csv = '/content/detect-ai-vs-human-generated-images/train.csv'
test_csv = '/content/detect-ai-vs-human-generated-images/test.csv'

# Load the training and test datasets
train = pd.read_csv(train_csv)
test = pd.read_csv(test_csv)

# Print dataset shapes
print(f'Training dataset shape: {train.shape}')
print(f'Test dataset shape: {test.shape}')

# Preprocess column names for consistency
train = train[['file_name', 'label']]
train.columns = ['id', 'label']

# Display columns for reference
print("Train columns:", train.columns)
print("Test columns:", test.columns)

Training dataset shape: (79950, 3)
Test dataset shape: (5540, 1)
Train columns: Index(['id', 'label'], dtype='object')
Test columns: Index(['id'], dtype='object')


In [ ]:
# Split the training data into training and validation sets (95% train, 5% validation)
train_df, val_df = train_test_split(
    train,
    test_size=0.05,
    random_state=42,
    stratify=train['label']
)

# Print shapes of the splits
print(f'Train shape: {train_df.shape}')
print(f'Validation shape: {val_df.shape}')

# Check class distribution in both sets
print("\nTrain class distribution:")
print(train_df['label'].value_counts(normalize=True))

print("\nValidation class distribution:")
print(val_df['label'].value_counts(normalize=True))

Train shape: (75952, 2)
Validation shape: (3998, 2)

Train class distribution:
label
0    0.5
1    0.5
Name: proportion, dtype: float64

Validation class distribution:
label
0    0.5
1    0.5
Name: proportion, dtype: float64


In [ ]:
# Training augmentations
train_transforms = transforms.Compose([
    transforms.Resize(232),  # Resize to match ConvNeXt preprocessing
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validation and Test transforms
val_test_transforms = transforms.Compose([
    transforms.Resize(232),  # Resize to 232 as per ConvNeXt documentation
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Dataset class for training and validation
class AIImageDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.dataframe.iloc[idx, 1]
        return image, label

# Dataset class for inference (validation and test)
class TestAIImageDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, os.path.basename(img_path)  # Return image and filename

In [ ]:
# Create datasets
train_dataset = AIImageDataset(train_df, root_dir=path, transform=train_transforms)

# For validation, create a list of file paths and store labels separately
val_file_list = [os.path.join(path, fname) for fname in val_df['id']]
val_labels = val_df['label'].values  # Store labels separately for later use
val_dataset = TestAIImageDataset(file_list=val_file_list, transform=val_test_transforms)

# For testing, create a list of file paths
test_file_list = [os.path.join(path, fname) for fname in test['id']]
test_dataset = TestAIImageDataset(file_list=test_file_list, transform=val_test_transforms)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

Training dataset size: 75952
Validation dataset size: 3998
Test dataset size: 5540


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Load pretrained ConvNeXt Base model
model = models.convnext_base(weights="DEFAULT")

# Freeze all layers initially
for param in model.features.parameters():
    param.requires_grad = False

# Unfreeze the last two stages
for param in model.features[-2:].parameters():
    param.requires_grad = True

# Replace the classifier head with a custom one
model.classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),  # Global average pooling
    nn.Flatten(),                  # Flatten the tensor
    nn.BatchNorm1d(1024),          # Add BatchNorm here
    nn.Linear(1024, 512),          # First fully connected layer
    nn.ReLU(),                     # Activation function
    nn.Dropout(0.4),               # Dropout for regularization
    nn.Linear(512, 2)              # Output layer (binary classification)
)

# Move the model to gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function, optimizer, and learning rate scheduler
optimizer = torch.optim.AdamW([
    {'params': model.features[-2:].parameters(), 'lr': 1e-5},  # Lower LR for backbone
    {'params': model.classifier.parameters(), 'lr': 1e-4}      # Higher LR for classifier
])

criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=5, gamma=0.7)

Downloading: "https://download.pytorch.org/models/convnext_base-6075fbad.pth" to /root/.cache/torch/hub/checkpoints/convnext_base-6075fbad.pth
100%|██████████| 338M/338M [00:02<00:00, 149MB/s]


In [ ]:
print("Training Start")

Training Start


In [ ]:

# Training Loop
epochs = 12

train_losses, train_accuracies, val_losses, val_accuracies, val_f1s = [], [], [], [], []

for epoch in range(epochs):
    # -- Training --
    model.train()
    epoch_loss = 0.0
    epoch_accuracy = 0.0

    for data, label in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        data, label = data.to(device), label.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        preds = output.argmax(dim=1)
        acc = (preds == label).float().mean().item()
        epoch_accuracy += acc

    epoch_loss /= len(train_loader)
    epoch_accuracy /= len(train_loader)

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    # -- Validation --
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    val_pred_classes = []  # To store predictions
    val_labels_list = []   # To store true labels

    with torch.no_grad():
        for i, (data, _) in enumerate(tqdm(val_loader, desc=f"Validation Epoch {epoch+1}")):
            data = data.to(device)
            output = model(data)

            # Get true labels from val_df
            batch_labels = val_labels[i * val_loader.batch_size : (i + 1) * val_loader.batch_size]
            batch_labels = torch.tensor(batch_labels, device=device)

            # Compute loss
            loss = criterion(output, batch_labels)
            val_loss += loss.item()

            # Compute predictions and accuracy
            preds = output.argmax(dim=1)
            acc = (preds == batch_labels).float().mean().item()
            val_acc += acc

            # Store predictions and true labels
            val_pred_classes.extend(preds.cpu().numpy())
            val_labels_list.extend(batch_labels.cpu().numpy())

    # Compute average validation metrics
    val_loss /= len(val_loader)
    val_acc /= len(val_loader)
    val_f1 = f1_score(val_labels_list, val_pred_classes, average='binary')  # Binary classification

    # Append metrics
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    val_f1s.append(val_f1)

    print(
        f"Epoch [{epoch+1}/{epochs}] "
        f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_accuracy:.4f} | "
        f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}"
    )

    # Step the learning rate scheduler
    scheduler.step()

Training Epoch 1:  32%|███▏      | 768/2374 [5:03:30<10:34:40, 23.71s/it]


KeyboardInterrupt: 

In [ ]:
# Generate predictions and logits for the test set
model.eval()
test_logits = []  # To store logits
test_pred_classes = []

with torch.no_grad():
    for data, _ in tqdm(test_loader, desc="Generating Test Predictions"):
        data = data.to(device)
        output = model(data)  # Raw logits (before softmax)

        # Save logits
        test_logits.extend(output.cpu().numpy())  # Store raw logits

        # Get predicted class (0 or 1)
        preds = output.argmax(dim=1)
        test_pred_classes.extend(preds.cpu().numpy())

# Convert logits to a DataFrame
logits_df = pd.DataFrame(test_logits, columns=['logit_class_0', 'logit_class_1'])
logits_df['id'] = test['id'].values  # Add image IDs for reference

# Save logits to a CSV file
logits_df.to_csv('test_logits.csv', index=False)

# Add predictions to the test DataFrame
test['label'] = test_pred_classes
test[['id', 'label']].to_csv('submission.csv', index=False)

print("Test logits saved to 'test_logits.csv'")
print("Test predictions saved to 'submission.csv'")

In [ ]:
# # Generate predictions for the test set
# model.eval()
# test_pred_classes = []

# with torch.no_grad():
#     for data, _ in tqdm(test_loader, desc="Generating Test Predictions"):
#         data = data.to(device)
#         output = model(data)
#         preds = output.argmax(dim=1)  # Get predicted class (0 or 1)
#         test_pred_classes.extend(preds.cpu().numpy())

# # Add predictions to the test DataFrame
# test['label'] = test_pred_classes

# # Save predictions to a CSV file
# test[['id', 'label']].to_csv('submission.csv', index=False)
# print("Test predictions saved to 'submission.csv'")

In [ ]:
pd.read_csv('submission.csv')['label'].value_counts()